In [1]:
import pandas as pd
df=pd.read_csv("/content/heart(in).csv")
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [2]:
df.shape

(918, 12)

In [3]:
df.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


In [4]:
df.isnull().sum()

,0
Age,0
Sex,0
ChestPainType,0
RestingBP,0
Cholesterol,0
FastingBS,0
RestingECG,0
MaxHR,0
ExerciseAngina,0
Oldpeak,0


In [5]:
df['ChestPainType'].unique()

array(['ATA', 'NAP', 'ASY', 'TA'], dtype=object)

In [6]:
df['RestingECG'].unique()

array(['Normal', 'ST', 'LVH'], dtype=object)

In [7]:
df['Sex'] = df['Sex'].map({'M': 0, 'F': 1})
df['ChestPainType']=df['ChestPainType'].map({'ATA':0,'NAP':1,'ASY':2,'TA':3})
df['RestingECG']=df['RestingECG'].map({'Normal':0,'ST':1,'LVH':2})
df['ExerciseAngina']=df['ExerciseAngina'].map({'N':0,'Y':1})
df['ST_Slope']=df['ST_Slope'].map({'Up':0,'Flat':1,'Down':2})

In [8]:
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,0,0,140,289,0,0,172,0,0.0,0,0
1,49,1,1,160,180,0,0,156,0,1.0,1,1
2,37,0,0,130,283,0,1,98,0,0.0,0,0
3,48,1,2,138,214,0,0,108,1,1.5,1,1
4,54,0,1,150,195,0,0,122,0,0.0,0,0


In [9]:
from sklearn.preprocessing import MinMaxScaler

X = df.drop('HeartDisease', axis=1)
y = df['HeartDisease']

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)


In [10]:
df.shape

(918, 12)

In [11]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [12]:
from keras import Sequential
from keras.layers import Dense,Dropout
from keras. callbacks import EarlyStopping,ModelCheckpoint

model = Sequential()
model.add(Dense(units=16, activation='relu',input_dim=11,kernel_initializer='random_uniform'))
model.add(Dense(32,kernel_initializer='random_uniform', activation='relu'))
model.add(Dense(40,kernel_initializer='random_uniform', activation='relu'))
model.add(Dense(32,kernel_initializer='random_uniform', activation='relu'))
model.add(Dense(16,kernel_initializer='random_uniform', activation='relu'))
model.add(Dense(1, activation='sigmoid'))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

early_stop= EarlyStopping(monitor='val_loss',
                               patience=3,
                               verbose=1,
                               restore_best_weights=True)

model_checkpoint = ModelCheckpoint('model1.keras',
                                   monitor='val_loss',
                                   verbose=1,
                                   save_best_only=True)

In [14]:
model.fit(X_train, y_train, epochs=50,
          batch_size=32,
          validation_split=0.2,
          verbose=1,
          callbacks=[early_stop, model_checkpoint])

Epoch 1/50
15/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5477 - loss: 0.6927 
Epoch 1: val_loss improved from inf to 0.69344, saving model to model1.keras
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 35ms/step - accuracy: 0.5492 - loss: 0.6926 - val_accuracy: 0.4966 - val_loss: 0.6934
Epoch 2/50
18/19 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5966 - loss: 0.6890
Epoch 2: val_loss improved from 0.69344 to 0.69335, saving model to model1.keras
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.5928 - loss: 0.6892 - val_accuracy: 0.4966 - val_loss: 0.6933
Epoch 3/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5542 - loss: 0.6867
Epoch 3: val_loss improved from 0.69335 to 0.68368, saving model to model1.keras
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.5544 - loss: 0.6866 - val_accuracy: 0.4966 - val_loss: 0.6837
Epoch 4/50
11/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5700 - loss: 0.6628  
Epoch 4: val_loss improved from 0.68368 to 0.61841, saving model to model1

In [15]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 16)             │           192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │           544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 40)             │         1,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 32)             │         1,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,741 (45.87 KB)

 Trainable params: 3,913 (15.29 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 7,828 (30.58 KB)

In [17]:
from sklearn.metrics import accuracy_score

loss,accuracy=model.evaluate(X_test,y_test)
print(f"Test Accuracy:{accuracy:.4f}")

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8168 - loss: 0.4231 
Test Accuracy:0.8315


In [22]:
loss

0.40654414892196655

In [23]:
accuracy

0.83152174949646

In [26]:
model.predict(X_test)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 


array([[0.04718928],
       [0.23803346],
       [0.8962623 ],
       [0.9280888 ],
       [0.06646325],
       [0.7800838 ],
       [0.79434836],
       [0.09265017],
       [0.8443453 ],
       [0.9568319 ],
       [0.8132628 ],
       [0.20552719],
       [0.40724683],
       [0.0640602 ],
       [0.8395783 ],
       [0.49927658],
       [0.05834012],
       [0.92569315],
       [0.7530724 ],
       [0.07102582],
       [0.8686655 ],
       [0.82365465],
       [0.07334428],
       [0.8419801 ],
       [0.9429691 ],
       [0.88984376],
       [0.0678327 ],
       [0.9366549 ],
       [0.06271525],
       [0.06558361],
       [0.7322545 ],
       [0.08286954],
       [0.94598866],
       [0.8997919 ],
       [0.95230734],
       [0.81470823],
       [0.89439225],
       [0.07070465],
       [0.8840567 ],
       [0.79013896],
       [0.82246995],
       [0.79320264],
       [0.8951785 ],
       [0.09328563],
       [0.09319229],
       [0.7555457 ],
       [0.48808625],
       [0.929

In [20]:
import numpy as np
y_pred_prob = model.predict(X_test)
y_pred = np.round(y_pred_prob).flatten()

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 


In [21]:
accuracy_score=accuracy_score(y_test,y_pred)
print(f"Test Accuracy:{accuracy_score:.4f}")

Test Accuracy:0.8315


In [12]:
from keras import Sequential
from keras.layers import Dense,Dropout
from keras. callbacks import EarlyStopping,ModelCheckpoint

model2 = Sequential()
model2.add(Dense(units=16, activation='relu',input_dim=11,kernel_initializer='random_uniform'))
model2.add(Dense(32,kernel_initializer='random_uniform', activation='relu'))
model2.add(Dropout(0.1))
model2.add(Dense(64,kernel_initializer='random_uniform', activation='relu'))
model2.add(Dropout(0.1))
model2.add(Dense(32,kernel_initializer='random_uniform', activation='relu'))
model2.add(Dense(16,kernel_initializer='random_uniform', activation='relu'))
model2.add(Dense(1, activation='sigmoid'))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['recall'])

early_stop2= EarlyStopping(monitor='val_loss',
                               patience=3,
                               verbose=1,
                               restore_best_weights=True)

model_checkpoint2 = ModelCheckpoint('model1.keras',
                                   monitor='val_loss',
                                   verbose=1,
                                   save_best_only=True)

In [16]:
model2.fit(X_train, y_train, epochs=50,
          batch_size=32,
          validation_split=0.2,
          verbose=1,
          callbacks=[early_stop2, model_checkpoint2])

Epoch 1/50
 1/19 ━━━━━━━━━━━━━━━━━━━━ 48s 3s/step - loss: 0.6932 - recall: 1.0000
Epoch 1: val_loss improved from inf to 0.69283, saving model to model1.keras
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - loss: 0.6915 - recall: 1.0000 - val_loss: 0.6928 - val_recall: 1.0000
Epoch 2/50
 1/19 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.6958 - recall: 1.0000
Epoch 2: val_loss improved from 0.69283 to 0.68508, saving model to model1.keras
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.6876 - recall: 1.0000 - val_loss: 0.6851 - val_recall: 1.0000
Epoch 3/50
 1/19 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.6900 - recall: 1.0000
Epoch 3: val_loss improved from 0.68508 to 0.63001, saving model to model1.keras
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.6643 - recall: 1.0000 - val_loss: 0.6300 - val_recall: 1.0000
Epoch 4/50
16/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5717 - recall: 0.9986  
Epoch 4: val_loss improved from 0.63001 to 0.57307, saving model to model1.keras
19/19 ━━━━━━